In [ ]:
# how many data records to process
sample_size = 200

# Load packages

In [ ]:
import pandas as pd
import numpy as np
import time
#####################################
# connecting with Google Drive
import gspread
from google.colab import auth
auth.authenticate_user()
from google.auth import default

creds, _ = default()
gcolab = gspread.authorize(creds)

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
# Load spacy
import spacy

!python -m spacy download en_core_web_lg
# Load the English NLP model
nlp_spacy = spacy.load("en_core_web_lg")
#nlp = spacy.load("en_core_web_md")

2024-01-05 09:42:23.832507: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 09:42:23.832598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 09:42:23.834705: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 09:42:26.811486: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


# Load data

In [ ]:
# Load data from the second sheet
gsheets = gcolab.open_by_url('https://docs.google.com/spreadsheets/d/1mer2kkjM7u-jJGsN0OXr4pne8FEBmYVzFgNXAx34SYk/edit#gid=0')
second_sheet_title = 'Modern agriculture'  # Replace with the actual title of your second sheet
sheet = gsheets.worksheet(second_sheet_title)
data = sheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0]).set_index("doc_id")
df = df[["score_relevance", "title", "abstract", "doc_date", "cities"]]
df['doc_date'] = pd.to_datetime(df['doc_date'])

#####################################
# concatenate title and abstract
text = []
for index, row in df.iterrows():
  if row["abstract"].strip() != "":
    text.append(row["title"] + " | " + row["abstract"])
  else:
    text.append(row["title"])

df["text"] = text
del text

df.drop(['title','abstract'], axis = 1, inplace = True)

In [ ]:
#######################################
# reshape location data

####################################
# extract city names
import ast
import json
def extract_city_name(cities_json):
    try:
        cities_list = json.loads(cities_json)
        if isinstance(cities_list, list) and cities_list:
            return [city['city_name'] for city in cities_list]
    except (json.JSONDecodeError, IndexError):
        pass
    return ''

df['location_names_heuristics'] = df['cities'].apply(extract_city_name)

####################################
# extract geolocations
####################################
# extract geolocations
def extract_geolocations(cities_json):
    try:
        cities_list = json.loads(cities_json)
        if isinstance(cities_list, list) and cities_list:
            result = [{city["city_name"]: [city["lat_long"][0], city["lat_long"][1]]} for city in cities_list]
            result = {k: v for d in result for k, v in d.items()}
            return result
    except (json.JSONDecodeError, IndexError):
        pass
    return ''

df['geolocations_heuristics'] = df['cities'].apply(extract_geolocations)

df.drop(['cities'], axis = 1, inplace = True)

In [ ]:
# sample and inspect data
df1 = df.sample(n = sample_size, random_state = 123)
df1.head()

,score_relevance,doc_date,text,location_names_heuristics,geolocations_heuristics
doc_id,,,,,
942997905,0.3215,2022-05-09,XAG promotes drones in Vietnam to boost rice f...,,
747044041,0.7616,2021-05-30,Vertical Farming Produce Market will touch a n...,,
1018822786,0.1363,2022-09-13,Artificial Intelligence for Smart Cities and V...,,
1177842567,0.2449,2023-04-26,Internet-of-Things (IoT) in Precision Agricult...,,
1241316330,0.8854,2023-06-16,The second life of containers meets vertical f...,[Brescia],"{'Brescia': [45.53558, 10.21472]}"


# Apply BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenCl

In [ ]:
example = "Japan, South China Morning Post"
ner_results = nlp(example)

# Extract location entities
location_entities = [entity['word'] for entity in ner_results if entity['entity'] == 'B-LOC' or entity['entity'] == 'I-LOC']

In [ ]:
print("Location Entities:", location_entities)

Location Entities: ['Japan']


In [ ]:
start_time_BERT = time.time()

# Extract locations from 'text' column
df1['location_names_BERT'] = df1['text'].apply(lambda title: ' '.join([entity['word'] for entity in nlp(title) if entity['entity'] in ['B-LOC', 'I-LOC']]))

# Ensure length of 'location_names_BERT' matches the length of the DataFrame index
df1['location_names_BERT'] = df1['location_names_BERT'].apply(lambda x: x if x else '')  # Replace with a placeholder or an empty string

# Extract only locations from 'location_names_BERT' column
df1['location_names_BERT'] = df1['location_names_BERT'].apply(lambda title: ' '.join([entity['word'] for entity in nlp(title) if entity['entity'] in ['B-LOC', 'I-LOC']]))

# Ensure the length of 'location_names_BERT' matches the length of the DataFrame index
df1['location_names_BERT'] = df1['location_names_BERT'].apply(lambda x: x if x else '')  # Replace with a placeholder or an empty string

end_time_BERT = time.time()

print()
print("Time taken by BERT location extraction:", round(end_time_BERT - start_time_BERT, 1), "seconds")


Time taken by BERT location extraction: 146.3 seconds


In [ ]:
# If you want to replace empty strings with NaN
#df1['location_names_BERT'] = df1['location_names_BERT'].replace('', np.nan)

# Replace spaces with commas
df1['location_names_BERT'] = df1['location_names_BERT'].str.replace(' ', ',')

# Split the string into a list and convert it to a set to get unique values
df1['location_names_BERT'] = df1['location_names_BERT'].str.split(',').apply(set)

# Join the set back into a string
df1['location_names_BERT'] = df1['location_names_BERT'].apply(lambda x: ','.join(x))

# Replace '##' in the refined column
df1['location_names_BERT'] = df1['location_names_BERT'].str.replace('##', '')

# Remove leading and trailing commas
df1['location_names_BERT'] = df1['location_names_BERT'].str.strip(',')

In [ ]:
df1.head()

,score_relevance,doc_date,text,location_names_heuristics,geolocations_heuristics,location_names_BERT
doc_id,,,,,,
942997905,0.3215,2022-05-09,XAG promotes drones in Vietnam to boost rice f...,,,Vietnam
747044041,0.7616,2021-05-30,Vertical Farming Produce Market will touch a n...,,,
1018822786,0.1363,2022-09-13,Artificial Intelligence for Smart Cities and V...,,,
1177842567,0.2449,2023-04-26,Internet-of-Things (IoT) in Precision Agricult...,,,
1241316330,0.8854,2023-06-16,The second life of containers meets vertical f...,[Brescia],"{'Brescia': [45.53558, 10.21472]}",


# Apply spacy

In [ ]:
# Function to extract locations using Spacy
def extract_locations_spacy(text):
    doc = nlp_spacy(text)
    # Extract entities that are classified as 'GPE' (Geopolitical Entity) or 'LOC' (Location)
    locations = [ent.text for ent in doc.ents if ent.label_ in ['GPE']]
    return locations

In [ ]:
start_time_spacy = time.time()

# Apply the function to the 'text' column and store the results in a new column
df1['location_names_spacy'] = df1['text'].apply(extract_locations_spacy)

# Convert lists to sets to remove duplicates
df1['location_names_spacy'] = df1['location_names_spacy'].apply(lambda x: ','.join(list(set(x))))

df1['location_names_spacy'] = df1['location_names_spacy'].apply(extract_locations_spacy)

# Join the set back into a string
df1['location_names_spacy'] = df1['location_names_spacy'].apply(lambda x: ','.join(x))

#df1['location_names_spacy'] = df1['location_names_spacy'].apply(extract_locations_spacy)

end_time_spacy = time.time()

print()
print("Time taken by spacy location extraction:", round(end_time_spacy - start_time_spacy, 1), "seconds")


Time taken by spacy location extraction: 10.2 seconds


In [ ]:
# Display the first few rows to verify the extracted locations
df1[['text', 'location_names_heuristics', 'location_names_BERT', 'location_names_spacy']]

,text,location_names_heuristics,location_names_BERT,location_names_spacy
doc_id,,,,
942997905,XAG promotes drones in Vietnam to boost rice f...,,Vietnam,Vietnam
747044041,Vertical Farming Produce Market will touch a n...,,,
1018822786,Artificial Intelligence for Smart Cities and V...,,,
1177842567,Internet-of-Things (IoT) in Precision Agricult...,,,
1241316330,The second life of containers meets vertical f...,[Brescia],,Brescia
...,...,...,...,...
888125519,Robotics in agriculture | When a robot is ment...,,,
899149100,Startup news and updates: daily roundup (May 2...,,US,"US,India"
879064632,Agriculture Robots and Drone Market Covid-19 I...,,,


In [ ]:
#!pip install retrying

# Apply geocoding

In [ ]:
from geopy.geocoders import Nominatim
import geopy


# Set a custom user agent
geopy.geocoders.options.default_user_agent = "my-application"

# Assuming df1 is your DataFrame with a column 'location_names_spacy'
geolocator = Nominatim(user_agent="my-application_Vin")

def geocode_location(locations):
    result = {}
    if locations != '':
      for location in locations.split(','):
          try:
              geo = geolocator.geocode(location.strip())
              time.sleep(2)  # Add a delay of 2 seconds between requests
              result[location] = [geo.latitude, geo.longitude] if geo else None
          except Exception as e:
              print(f"Error geocoding {location}: {e}")
              result[location] = None
    return result

In [ ]:
geocode_location('canada, toronto')

{'canada': [61.0666922, -107.991707], ' toronto': [43.6534817, -79.3839347]}

In [ ]:
#start_time_BERT_geocoding = time.time()
#df1['geolocations_BERT'] = df1['location_names_BERT'].apply(geocode_location)
#end_time_BERT_geocoding = time.time()

start_time_spacy_geocoding = time.time()
df1['geolocations_spacy'] = df1['location_names_spacy'].apply(geocode_location)
end_time_spacy_geocoding = time.time()

#print("Time taken for geocoding (BERT):", end_time_BERT_geocoding - start_time_BERT_geocoding, "seconds")
#print()
print("Time taken for geocoding (Spacy):", end_time_spacy_geocoding - start_time_spacy_geocoding, "seconds")

Error geocoding Vietnam: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Vietnam&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Error geocoding UK: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=UK&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Error geocoding Portugal: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Portugal&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Time taken for geocoding (Spacy): 446.4853367805481 seconds


# Compare results

In [ ]:
df1[['text', 'geolocations_heuristics', 'geolocations_spacy']]

,text,geolocations_heuristics,geolocations_spacy
doc_id,,,
942997905,XAG promotes drones in Vietnam to boost rice f...,,{'Vietnam': None}
747044041,Vertical Farming Produce Market will touch a n...,,{}
1018822786,Artificial Intelligence for Smart Cities and V...,,{}
1177842567,Internet-of-Things (IoT) in Precision Agricult...,,{}
1241316330,The second life of containers meets vertical f...,"{'Brescia': [45.53558, 10.21472]}","{'Brescia': [45.77958045, 10.4258729694612]}"
...,...,...,...
888125519,Robotics in agriculture | When a robot is ment...,,{}
899149100,Startup news and updates: daily roundup (May 2...,,"{'US': [17.789187, -64.7080574], 'India': [22...."
879064632,Agriculture Robots and Drone Market Covid-19 I...,,{}


# Combine spacy + heuristics

In [ ]:
locations_final = []
for index, row in df1.iterrows():
  filter = row["geolocations_spacy"]
  filter = list(filter.keys())

  locations = row["geolocations_heuristics"]
  if locations == '':
    locations = {}

  filter = list(set(filter) & set(locations.keys()))
  locations = [{key: locations[key]} for key in filter]
  locations = {k: v for d in locations for k, v in d.items()}
  locations_final.append(locations)

df1["locations_final"] = locations_final

In [ ]:
df1[["text", "locations_final"]][df1["locations_final"] != {}]

,text,locations_final
doc_id,,
1241316330,The second life of containers meets vertical f...,"{'Brescia': [45.53558, 10.21472]}"
1026835683,Tauranga firm's robot cut above rest at US agr...,"{'Tauranga': [-37.68611, 176.16667]}"
1223848976,The Power of AI in Food Science | AI is transf...,"{'Shanghai': [31.22222, 121.45806]}"
1223847270,"Three industries ripe for automation, accordin...","{'Detroit': [42.33143, -83.04575], 'Pittsburgh..."
985344638,"Exclusive Interview with Kalyan Enjamoori, CTO...","{'Hyderabad': [17.38405, 78.45636]}"
1257448538,Chief commercial officer named at vertical far...,"{'Jersey City': [40.72816, -74.07764], 'Manhat..."
1265329247,IFA 2023: Can new consumer tech still excite? ...,"{'Berlin': [52.52437, 13.41053]}"
1083759828,FarmBot Ships Record Number of Farming Robots ...,"{'Bakersfield': [35.37329, -119.01871]}"
1225205898,Eleven ag leaders to be inducted into Ontario ...,"{'Ontario': [34.06334, -117.65089]}"
